In [42]:
#import statemets

from customScripts import utilities as util
from customScripts import features as feat

In [43]:
# parameters

file_path = 'audio_files/Muppets-02-01-01.wav'

#number of samples / second
sampling_rate = 48000

#length of frame in samples
frame_length = 48000

#number of samples used as offset for earch consecutive frame
hop_length = 24000

#number of mel frequency bins to use
bin_number = 40 # also try 80

#set the desired number of frames / second here
ground_thruth_conversion_const = 1

In [44]:
#load audio files

y = util.load_audio(file_path, sampling_rate)

In [45]:
#compute spectrogram

spectrogram = feat.compute_spectrogram(y, frame_length, hop_length, bin_number)